Лабораторна робота №6 Тема: Рекомендаційні системи

In [4]:
!pip install scikit-surprise
from surprise import Dataset
from surprise import Reader
import pandas as pd
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate

In [3]:
# Завдання 1
# 1.Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise

data = Dataset.load_builtin('ml-100k')
df = pd.DataFrame(data.raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])

# 2.Виведіть перші 5 рядків завантаженого датасету
df.head()

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [6]:
# 3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
# датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.

algo_svd = SVD()
algo_knn = KNNBasic()

In [7]:
# 4. Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.

cv_svd = cross_validate(algo_svd, data, measures=['MAE'], cv=5, verbose=True)
cv_knn = cross_validate(algo_knn, data, measures=['MAE'], cv=5, verbose=True)

Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7403  0.7347  0.7392  0.7386  0.7329  0.7371  0.0028  
Fit time          1.71    1.75    1.36    1.36    1.37    1.51    0.18    
Test time         0.22    0.26    0.11    0.23    0.17    0.20    0.05    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7765  0.7671  0.7772  0.7744  0.7702  0.7731  0.0038  
Fit time          0.37    0.48    0.38    0.38    0.51    0.42    0.06    
Test time

In [8]:
# 5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки (MAE).
mae_svd = sum(cv_svd['test_mae']) / len(cv_svd['test_mae'])
mae_knn = sum(cv_knn['test_mae']) / len(cv_knn['test_mae'])

print(f"MAE для SVD: {mae_svd}")
print(f"MAE для KNN: {mae_knn}")

best_algo = algo_svd if mae_svd < mae_knn else algo_knn
print(f"Найкращий алгоритм: {'SVD' if best_algo == algo_svd else 'KNNBasic'}")


MAE для SVD: 0.7371419536318884
MAE для KNN: 0.7730656148361212
Найкращий алгоритм: SVD


In [9]:
# 6. Виведи рекомендації (10 фільмів) для конкретного користувача.
trainset = data.build_full_trainset()
best_algo.fit(trainset)

user_id = str(100) # Було обрано користувача під номером 100
items = df['item'].unique()

items_to_rate = [item for item in items if not trainset.knows_item(item)]
predictions = [best_algo.predict(user_id, item) for item in items_to_rate]
recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

print("Рекомендовані фільми для користувача №100:")
for rec in recommendations:
    print(f"Фільм ID: {rec.iid}, Прогнозований рейтинг: {rec.est}")


Рекомендовані фільми для користувача №100:
Фільм ID: 313, Прогнозований рейтинг: 4.406179281805028
Фільм ID: 169, Прогнозований рейтинг: 4.266908829497051
Фільм ID: 318, Прогнозований рейтинг: 4.101961646130496
Фільм ID: 483, Прогнозований рейтинг: 4.099608321606012
Фільм ID: 316, Прогнозований рейтинг: 4.068332960235825
Фільм ID: 515, Прогнозований рейтинг: 4.049758213176435
Фільм ID: 272, Прогнозований рейтинг: 4.044792071249328
Фільм ID: 496, Прогнозований рейтинг: 4.038463608581213
Фільм ID: 480, Прогнозований рейтинг: 4.03086074075311
Фільм ID: 923, Прогнозований рейтинг: 4.0270327167892495


In [10]:
# Завдання 2
# 1. Побудуйте власну рекомендаційну систему (приклад є в документації). Оцініть її.

from collections import defaultdict

def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [12]:
# 2. Отримайте рекомендацію для певного користувача.

trainset = data.build_full_trainset()
best_algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = best_algo.test(testset)

top_n = get_top_n(predictions, n=10)

print(f"Рекомендації для користувача №100: {top_n['100']}")


Рекомендації для користувача №100: [('169', 4.284224613657105), ('408', 4.223965456085339), ('483', 4.194623751291536), ('515', 4.179746724648006), ('923', 4.1338357693871925), ('170', 4.114678580293429), ('22', 4.114426346737091), ('511', 4.109287018610404), ('496', 4.068186009167999), ('64', 4.061903707952747)]


Висновок: У ході виконання лабораторної роботи було досліджено використання алгоритмів із бібліотеки Surprise та методи розробки власної рекомендаційної системи. Було проаналізовано алгоритми SVD та KNN, проведена оцінка точності за допомогою метрики MAE. Також на основі отриманих даних було побудовано список рекомендацій для конкретного користувача. Використання обох алгоритмів допомгло з'ясувати, що вони є ефективними методами вирішення задач рекомендаційних систем. Результати роботи були завантажені на відповідний Github-репозиторій.